In [5]:
import Utils.PTModel.Training as Training
import Utils.PTModel.Inference as Inference
import Utils.Data.DataLoading as DataLoading
import Utils.PTModel.Models as Models

import Utils.EvaluationMetrics.alphaScore as AS
import Utils.EvaluationMetrics.exampleBased as EB
import Utils.EvaluationMetrics.labelBased as LB

import numpy as np

In [3]:
crossValidationData = DataLoading.LoadCrossValTrainTestData("../data/tomData/unshuffled3x3tiles.csv", shuffle=True)

In [8]:
print(f"--- Cross Validation Testing ---")

for k, v in crossValidationData.items():
    print(f"\n{k}")
    print(f"trainDataShape: {v['trainData'].shape} | testDataShape: {v['testData'].shape}")

    model = Training.TrainModel(v, 1, 32, Models.TileEmbeddingVAE)

    yPredImage, yPredText = Inference.ModelInference(model, v['testData'])
    yTruthText = np.array(v['testData']["encodedAffordances"].to_list())

    print(f"\nAlpha Score beta = 0.75 | gamma = 0.25: {AS.alpha_score(yTruthText, yPredText, beta=0.75, gamma=0.25)}")
    print(f"Alpha Score beta = 1.0 | gamma = 1.0: {AS.alpha_score(yTruthText, yPredText, beta=1.0, gamma=1.0)}")
    print(f"Alpha Score beta = 0.25 | gamma = 0.75: {AS.alpha_score(yTruthText, yPredText, beta=0.25, gamma=0.75)}")

    print(f"\nExample Based Accuracy: {EB.example_based_accuracy(yTruthText, yPredText)}")
    print(f"Example Based Precision: {EB.example_based_precision(yTruthText, yPredText)}")
    print(f"Example Based Recall: {EB.example_based_recall(yTruthText, yPredText)}")
    print(f"Example Based EMR: {EB.hamming_loss(yTruthText, yPredText)}")

    # print("\nMicro")
    # print(f"Label Based Accuracy: {LB.accuracy_micro(yTruthText, yPredText)}")
    # print(f"Label Based Precision: {LB.precision_micro(yTruthText, yPredText)}")
    # print(f"Label Based Recall: {LB.recall_micro(yTruthText, yPredText)}")

    print("\nMacro")
    print(f"Label Based Accuracy: {LB.accuracy_macro(yTruthText, yPredText)}")
    print(f"Label Based Precision: {LB.precision_macro(yTruthText, yPredText)}")
    print(f"Label Based Recall: {LB.recall_macro(yTruthText, yPredText)}")

--- Cross Validation Testing For Model ---

loderunner
trainDataShape: (4841, 7) | testDataShape: (15151, 7)
Epoch 0: loss 9618.817675138775

Alpha Score beta = 0.75 | gamma = 0.25: 0.574839629365645
Alpha Score beta = 1.0 | gamma = 1.0: 0.36130651709070627
Alpha Score beta = 0.25 | gamma = 0.75: 0.7864668877250612

Example Based Accuracy: 0.43455877499834994
Example Based Precision: 0.7575737575077552
Example Based Recall: 0.43455877499834994
Example Based EMR: 0.10463894233942415

Micro
Label Based Accuracy: 0.3613065183162689
Label Based Precision: 0.7703336477279663
Label Based Recall: 0.40492480993270874

Macro
Label Based Accuracy: 0.09770769339341384
Label Based Precision: 0.20206368886507475
Label Based Recall: 0.12619686126708984

thelegendofzelda
trainDataShape: (18362, 7) | testDataShape: (1630, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/exampleBased.py:34: RuntimeWarning: invalid value encountered in divide
  avg_precision=np.nansum(p_n/p_d)/float(n)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:16: RuntimeWarning: invalid value encountered in divide
  sum_acc=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:31: RuntimeWarning: invalid value encountered in divide
  sum_precision=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


Epoch 0: loss 6286.523001534598

Alpha Score beta = 0.75 | gamma = 0.25: 0.8273898711417441
Alpha Score beta = 1.0 | gamma = 1.0: 0.36050344620916996
Alpha Score beta = 0.25 | gamma = 0.75: 0.5331135750674258

Example Based Accuracy: 0.36779141104294477
Example Based Precision: 0.3711349768141296
Example Based Recall: 0.947239263803681
Example Based EMR: 0.20141576215195847

Micro
Label Based Accuracy: 0.3605034351348877
Label Based Precision: 0.36992618441581726
Label Based Recall: 0.9340062141418457

Macro
Label Based Accuracy: 0.24638119110694298
Label Based Precision: 0.2808660360483023
Label Based Recall: 0.48284097818227917

megaman
trainDataShape: (18523, 7) | testDataShape: (1469, 7)
Epoch 0: loss 6079.512948729625

Alpha Score beta = 0.75 | gamma = 0.25: 0.8274262075134168
Alpha Score beta = 1.0 | gamma = 1.0: 0.44879248658318427
Alpha Score beta = 0.25 | gamma = 0.75: 0.6213662790697674

Example Based Accuracy: 0.5860788670523185
Example Based Precision: 0.6012478926459326
Ex